In [3]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

In [4]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)